In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os
import glob
import PIL
from PIL import Image
from torch.utils import data as D
from torch.utils.data.sampler import SubsetRandomSampler
import random

In [2]:
batch_size = 32
validation_ratio = 0.1
random_seed = 10

In [3]:
transform_train = transforms.Compose([
        transforms.Resize(299),
        transforms.RandomCrop(299, padding=38),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

transform_validation = transforms.Compose([
        transforms.Resize(299),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])


transform_test = transforms.Compose([
        transforms.Resize(299),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)

validset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_validation)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)

#trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                          shuffle=True, num_workers=0)

num_train = len(trainset)
indices = list(range(num_train))
split = int(np.floor(validation_ratio * num_train))

np.random.seed(random_seed)
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, sampler=train_sampler, num_workers=0
)

valid_loader = torch.utils.data.DataLoader(
    validset, batch_size=batch_size, sampler=valid_sampler, num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=0
)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

initial_lr = 0.045

100%|██████████| 170498071/170498071 [00:03<00:00, 46444069.01it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


## Reduced Dataset

In [4]:
import torchvision.transforms as transforms
import torch

included_classes = ['plane', 'car', 'bird', 'cat', 'deer']
# included_classes = ['plane', 'car']


transform_train = transforms.Compose([
        transforms.Resize(299),
        transforms.RandomCrop(299, padding=38),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

transform_validation = transforms.Compose([
        transforms.Resize(299),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])


transform_test = transforms.Compose([
        transforms.Resize(299),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainset = torch.utils.data.Subset(trainset, [idx for idx in range(len(trainset)) if trainset.targets[idx] < len(included_classes)])

validset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_validation)
validset = torch.utils.data.Subset(validset, [idx for idx in range(len(validset)) if validset.targets[idx] < len(included_classes)])

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testset = torch.utils.data.Subset(testset, [idx for idx in range(len(testset)) if testset.targets[idx] < len(included_classes)])

num_train = len(trainset)
indices = list(range(num_train))
split = int(np.floor(validation_ratio * num_train))

np.random.seed(random_seed)
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, sampler=train_sampler, num_workers=0
)

valid_loader = torch.utils.data.DataLoader(
    validset, batch_size=batch_size, sampler=valid_sampler, num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=0
)

classes = ('plane', 'car', 'bird', 'cat', 'deer')

initial_lr = 0.045

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Proposed Paper Xception Model

In [5]:
class depthwise_separable_conv(nn.Module):
    def __init__(self, nin, nout, kernel_size, padding, bias=False):
        super(depthwise_separable_conv, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin, kernel_size=kernel_size, padding=padding, groups=nin, bias=bias)
        self.pointwise = nn.Conv2d(nin, nout, kernel_size=1, bias=bias)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

In [6]:
class Xception(nn.Module):
    def __init__(self, input_channel, num_classes=10):
        super(Xception, self).__init__()
        
        # Entry Flow
        self.entry_flow_1 = nn.Sequential(
            nn.Conv2d(input_channel, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )
        
        self.entry_flow_2 = nn.Sequential(
            depthwise_separable_conv(64, 128, 3, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            depthwise_separable_conv(128, 128, 3, 1),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.entry_flow_2_residual = nn.Conv2d(64, 128, kernel_size=1, stride=2, padding=0)
        
        self.entry_flow_3 = nn.Sequential(
            nn.ReLU(True),
            depthwise_separable_conv(128, 256, 3, 1),
            nn.BatchNorm2d(256),
            
            nn.ReLU(True),
            depthwise_separable_conv(256, 256, 3, 1),
            nn.BatchNorm2d(256),
            
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.entry_flow_3_residual = nn.Conv2d(128, 256, kernel_size=1, stride=2, padding=0)
        
        self.entry_flow_4 = nn.Sequential(
            nn.ReLU(True),
            depthwise_separable_conv(256, 728, 3, 1),
            nn.BatchNorm2d(728),
            
            nn.ReLU(True),
            depthwise_separable_conv(728, 728, 3, 1),
            nn.BatchNorm2d(728),
            
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.entry_flow_4_residual = nn.Conv2d(256, 728, kernel_size=1, stride=2, padding=0)
        
        # Middle Flow
        self.middle_flow = nn.Sequential(
            nn.ReLU(True),
            depthwise_separable_conv(728, 728, 3, 1),
            nn.BatchNorm2d(728),
            
            nn.ReLU(True),
            depthwise_separable_conv(728, 728, 3, 1),
            nn.BatchNorm2d(728),
            
            nn.ReLU(True),
            depthwise_separable_conv(728, 728, 3, 1),
            nn.BatchNorm2d(728)
        )
        
        # Exit Flow
        self.exit_flow_1 = nn.Sequential(
            nn.ReLU(True),
            depthwise_separable_conv(728, 728, 3, 1),
            nn.BatchNorm2d(728),
            
            nn.ReLU(True),
            depthwise_separable_conv(728, 1024, 3, 1),
            nn.BatchNorm2d(1024),
            
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.exit_flow_1_residual = nn.Conv2d(728, 1024, kernel_size=1, stride=2, padding=0)
        self.exit_flow_2 = nn.Sequential(
            depthwise_separable_conv(1024, 1536, 3, 1),
            nn.BatchNorm2d(1536),
            nn.ReLU(True),
            
            depthwise_separable_conv(1536, 2048, 3, 1),
            nn.BatchNorm2d(2048),
            nn.ReLU(True)
        )
        
        self.linear = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        entry_out1 = self.entry_flow_1(x)
        entry_out2 = self.entry_flow_2(entry_out1) + self.entry_flow_2_residual(entry_out1)
        entry_out3 = self.entry_flow_3(entry_out2) + self.entry_flow_3_residual(entry_out2)
        entry_out = self.entry_flow_4(entry_out3) + self.entry_flow_4_residual(entry_out3)
        
        middle_out = self.middle_flow(entry_out) + entry_out
        
        for i in range(7):
          middle_out = self.middle_flow(middle_out) + middle_out

        exit_out1 = self.exit_flow_1(middle_out) + self.exit_flow_1_residual(middle_out)
        exit_out2 = self.exit_flow_2(exit_out1)

        exit_avg_pool = F.adaptive_avg_pool2d(exit_out2, (1, 1))                
        exit_avg_pool_flat = exit_avg_pool.view(exit_avg_pool.size(0), -1)

        output = self.linear(exit_avg_pool_flat)
        
        return output

In [7]:
net = Xception(3, 10) #ResNet-18

# IMPROVED XCEPTION MODEL V1

Using Batch Normalization and Factorized Convolutions

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ImprovedXception(nn.Module):
    def __init__(self, input_channel, num_classes=10):
        super(ImprovedXception, self).__init__()
        
        # Entry Flow
        self.entry_flow_1 = nn.Sequential(
            nn.Conv2d(input_channel, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )
        
        self.entry_flow_2 = nn.Sequential(
            depthwise_separable_conv(64, 128, 3, 1, use_batchnorm=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.entry_flow_2_residual = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=1, stride=2, padding=0, bias=False),
            nn.BatchNorm2d(128)
        )
        
        self.entry_flow_3 = nn.Sequential(
            depthwise_separable_conv(128, 256, 3, 1, use_batchnorm=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.entry_flow_3_residual = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=1, stride=2, padding=0, bias=False),
            nn.BatchNorm2d(256)
        )
        
        self.entry_flow_4 = nn.Sequential(
            depthwise_separable_conv(256, 728, 3, 1, use_batchnorm=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.entry_flow_4_residual = nn.Sequential(
            nn.Conv2d(256, 728, kernel_size=1, stride=2, padding=0, bias=False),
            nn.BatchNorm2d(728)
        )
        
        # Middle Flow
        self.middle_flow = nn.Sequential(
            *[depthwise_separable_conv(728, 728, 3, 1, use_batchnorm=True) for _ in range(8)]
        )
        
        # Exit Flow
        self.exit_flow_1 = nn.Sequential(
            depthwise_separable_conv(728, 1024, 3, 1, use_batchnorm=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.exit_flow_1_residual = nn.Sequential(
            nn.Conv2d(728, 1024, kernel_size=1, stride=2, padding=0, bias=False),
            nn.BatchNorm2d(1024)
        )
        self.exit_flow_2 = nn.Sequential(
            depthwise_separable_conv(1024, 1536, 3, 1, use_batchnorm=True),
            depthwise_separable_conv(1536, 2048, 3, 1, use_batchnorm=True)
        )
        
        self.linear = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        x = self.entry_flow_1(x)
        residual = self.entry_flow_2_residual(x)
        x = self.entry_flow_2(x)
        x = F.relu(x + residual)
        
        residual = self.entry_flow_3_residual(x)
        x = self.entry_flow_3(x)
        x = F.relu(x + residual)
        
        residual = self.entry_flow_4_residual(x)
        x = self.entry_flow_4(x)
        x = F.relu(x + residual)
        
        x = self.middle_flow(x)
        
        residual = self.exit_flow_1_residual(x)
        x = self.exit_flow_1(x)
        x = F.relu(x + residual)
        
        x = self.exit_flow_2(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

def depthwise_separable_conv(nin, nout, kernel_size, padding, use_batchnorm=False):
    layers = [
        nn.Conv2d(nin, nin, kernel_size=kernel_size, padding=padding, groups=nin, bias=not use_batchnorm),
        nn.BatchNorm2d(nin) if use_batchnorm else nn.Identity(),
        nn.ReLU(True),
        nn.Conv2d(nin, nout, kernel_size=1, bias=not use_batchnorm),
        nn.BatchNorm2d(nout) if use_batchnorm else nn.Identity(),
        nn.ReLU(True)
    ]
    return nn.Sequential(*layers)


In [9]:
net1 = ImprovedXception(3, 10)

# Imporoved Xception Model V2

Using asymmetric conv

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ImprovedXception1(nn.Module):
    def __init__(self, input_channel, num_classes=10):
        super(ImprovedXception1, self).__init__()
        
        # Entry Flow
        self.entry_flow_1 = nn.Sequential(
            nn.Conv2d(input_channel, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )
        
        self.entry_flow_2 = nn.Sequential(
            depthwise_separable_conv(64, 128, 3, 1, use_batchnorm=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.entry_flow_2_residual = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=1, stride=2, padding=0, bias=False),
            nn.BatchNorm2d(128)
        )
        
        self.entry_flow_3 = nn.Sequential(
            depthwise_separable_conv(128, 256, 3, 1, use_batchnorm=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.entry_flow_3_residual = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=1, stride=2, padding=0, bias=False),
            nn.BatchNorm2d(256)
        )
        
        self.entry_flow_4 = nn.Sequential(
            depthwise_separable_conv(256, 728, 3, 1, use_batchnorm=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.entry_flow_4_residual = nn.Sequential(
            nn.Conv2d(256, 728, kernel_size=1, stride=2, padding=0, bias=False),
            nn.BatchNorm2d(728)
        )
        
        # Middle Flow
        self.middle_flow = nn.Sequential(
            *[depthwise_separable_conv(728, 728, 3, 1, use_batchnorm=True) for _ in range(8)]
        )
        
        # Exit Flow
        self.exit_flow_1 = nn.Sequential(
            depthwise_separable_conv(728, 1024, 3, 1, use_batchnorm=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.exit_flow_1_residual = nn.Sequential(
            nn.Conv2d(728, 1024, kernel_size=1, stride=2, padding=0, bias=False),
            nn.BatchNorm2d(1024)
        )
        self.exit_flow_2 = nn.Sequential(
            depthwise_separable_conv(1024, 1536, 3, 1, use_batchnorm=True),
            depthwise_separable_conv(1536, 2048, 3, 1, use_batchnorm=True)
        )
        
        self.linear = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        x = self.entry_flow_1(x)
        residual = self.entry_flow_2_residual(x)
        x = self.entry_flow_2(x)
        x = F.relu(x + residual)
        
        residual = self.entry_flow_3_residual(x)
        x = self.entry_flow_3(x)
        x = F.relu(x + residual)
        
        residual = self.entry_flow_4_residual(x)
        x = self.entry_flow_4(x)
        x = F.relu(x + residual)
        
        x = self.middle_flow(x)
        
        residual = self.exit_flow_1_residual(x)
        x = self.exit_flow_1(x)
        x = F.relu(x + residual)
        
        x = self.exit_flow_2(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

def depthwise_separable_conv(nin, nout, kernel_size, padding, use_batchnorm=False):
    layers = [
        nn.Conv2d(nin, nin, kernel_size=(kernel_size, 1), padding=(padding, 0), groups=nin, bias=not use_batchnorm),
        nn.Conv2d(nin, nin, kernel_size=(1, kernel_size), padding=(0, padding), groups=nin, bias=not use_batchnorm),
        nn.BatchNorm2d(nin) if use_batchnorm else nn.Identity(),
        nn.ReLU(True),
        nn.Conv2d(nin, nout, kernel_size=1, bias=not use_batchnorm),
        nn.BatchNorm2d(nout) if use_batchnorm else nn.Identity(),
        nn.ReLU(True)
    ]
    return nn.Sequential(*layers)



In [11]:
net2 = ImprovedXception1(3, 10)

## Original Xception

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [13]:
net.to(device)

Xception(
  (entry_flow_1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (entry_flow_2): Sequential(
    (0): depthwise_separable_conv(
      (depthwise): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): depthwise_separable_conv(
      (depthwise): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128, bias=False)
      (pointwise): Conv2d(128, 128, 

### Improved V1

In [14]:
device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device1)

cuda:0


In [15]:
net1.to(device1)

ImprovedXception(
  (entry_flow_1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (entry_flow_2): Sequential(
    (0): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, 

## Improved V2

In [16]:
device2 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device2)

cuda:0


In [17]:
net2.to(device2)

ImprovedXception1(
  (entry_flow_1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (entry_flow_2): Sequential(
    (0): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), groups=64, bias=False)
      (1): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), groups=64, bias=False)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
      (4): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

# Original

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=initial_lr, momentum=0.9)

for epoch in range(25):  # 데이터셋을 수차례 반복합니다.
    if epoch == 0:
        lr = initial_lr
    elif epoch % 2 == 0 and epoch != 0:
        lr *= 0.94
        optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        """
        running_loss += loss.item()
        
        show_period = 250
        if i % show_period == show_period-1:    # print every "show_period" mini-batches
            print('[%d, %5d] loss: %.7f' %
                  (epoch + 1, i + 1, running_loss / show_period))
            running_loss = 0.0
        """
        
    #validation part
    correct = 0
    total = 0
    for i, data in enumerate(valid_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('[%d epoch] Accuracy of the network on the validation images: %d %%' % 
          (epoch, 100 * correct / total)
         )

print('Finished Training')

[0 epoch] Accuracy of the network on the validation images: 46 %
[1 epoch] Accuracy of the network on the validation images: 55 %
[2 epoch] Accuracy of the network on the validation images: 59 %
[3 epoch] Accuracy of the network on the validation images: 65 %
[4 epoch] Accuracy of the network on the validation images: 68 %
[5 epoch] Accuracy of the network on the validation images: 76 %
[6 epoch] Accuracy of the network on the validation images: 78 %
[7 epoch] Accuracy of the network on the validation images: 78 %
[8 epoch] Accuracy of the network on the validation images: 81 %
[9 epoch] Accuracy of the network on the validation images: 80 %
[10 epoch] Accuracy of the network on the validation images: 82 %
[11 epoch] Accuracy of the network on the validation images: 82 %
[12 epoch] Accuracy of the network on the validation images: 83 %


In [ ]:
dataiter = iter(test_loader)
images, labels = next(dataiter)
images, labels = images.to(device), labels.to(device)

outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(len(images))))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(5)) #changed from 10 to 5 classes
class_total = list(0. for i in range(5)) #changed from 10 to 5 classes
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
                
        for i in range(labels.shape[0]):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(5): #changed from 10 to 5 classes
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

# Improved V1

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net1.parameters(), lr=initial_lr, momentum=0.9)

for epoch in range(20):  # 데이터셋을 수차례 반복합니다.
    if epoch == 0:
        lr = initial_lr
    elif epoch % 2 == 0 and epoch != 0:
        lr *= 0.94
        optimizer = optim.SGD(net1.parameters(), lr=lr, momentum=0.9)
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device1), labels.to(device1)

        optimizer.zero_grad()

        outputs = net1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        """
        running_loss += loss.item()
        
        show_period = 250
        if i % show_period == show_period-1:    # print every "show_period" mini-batches
            print('[%d, %5d] loss: %.7f' %
                  (epoch + 1, i + 1, running_loss / show_period))
            running_loss = 0.0
        """
        
    #validation part
    correct = 0
    total = 0
    for i, data in enumerate(valid_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device1), labels.to(device1)
        outputs = net1(inputs)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('[%d epoch] Accuracy of the network on the validation images: %d %%' % 
          (epoch, 100 * correct / total)
         )

print('Finished Training')

[0 epoch] Accuracy of the network on the validation images: 62 %
[1 epoch] Accuracy of the network on the validation images: 67 %
[2 epoch] Accuracy of the network on the validation images: 72 %
[3 epoch] Accuracy of the network on the validation images: 75 %
[4 epoch] Accuracy of the network on the validation images: 76 %
[5 epoch] Accuracy of the network on the validation images: 76 %
[6 epoch] Accuracy of the network on the validation images: 80 %
[7 epoch] Accuracy of the network on the validation images: 80 %
[8 epoch] Accuracy of the network on the validation images: 81 %
[9 epoch] Accuracy of the network on the validation images: 83 %
[10 epoch] Accuracy of the network on the validation images: 84 %
[11 epoch] Accuracy of the network on the validation images: 84 %
[12 epoch] Accuracy of the network on the validation images: 85 %
[13 epoch] Accuracy of the network on the validation images: 86 %
[14 epoch] Accuracy of the network on the validation images: 85 %
[15 epoch] Accuracy 

In [19]:
dataiter = iter(test_loader)
images, labels = next(dataiter)
images, labels = images.to(device1), labels.to(device1)

outputs = net1(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(len(images))))

Predicted:    cat plane   car   cat   car plane  bird  deer  bird  deer plane  deer   cat  deer   car  deer plane   cat   cat   cat  bird   cat   cat  bird   car  bird   car  bird plane  bird   cat   cat


In [20]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device1), labels.to(device1)
        outputs = net1(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 87 %


In [21]:
class_correct = list(0. for i in range(5)) #changed from 10 to 2 classes
class_total = list(0. for i in range(5)) #changed from 10 to 2 classes
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device1), labels.to(device1)
        outputs = net1(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
                
        for i in range(labels.shape[0]):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(5): #changed from 10 to 2 classes
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 91 %
Accuracy of   car : 90 %
Accuracy of  bird : 84 %
Accuracy of   cat : 85 %
Accuracy of  deer : 83 %


In [27]:
import torch
import time
from sklearn.metrics import precision_score, recall_score, f1_score
correct = 0
total = 0
all_labels = []
all_predictions = []

# To measure inference time
start_time = time.time()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device1), labels.to(device1)
        
        # Forward pass to get outputs
        outputs = net1(images)
        _, predicted = torch.max(outputs.data, 1)

        # Collect all labels and predictions for precision, recall, F1 score calculation
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

        # Calculating correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate inference time
inference_time = time.time() - start_time

# Convert lists to numpy arrays for metric calculation
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

# Calculate precision, recall, and F1 score
precision = precision_score(all_labels, all_predictions, average='macro')
recall = recall_score(all_labels, all_predictions, average='macro')
f1 = f1_score(all_labels, all_predictions, average='macro')

print('Accuracy of the network on the test images: {:.2f} %'.format(100 * correct / total))
print('Precision: {:.4f}'.format(precision))
print('Recall: {:.4f}'.format(recall))
print('F1 Score: {:.4f}'.format(f1))
print('Inference Time: {:.2f} seconds'.format(inference_time))

Accuracy of the network on the test images: 87.14 %
Precision: 0.8752
Recall: 0.8714
F1 Score: 0.8722
Inference Time: 23.53 seconds


# Imporved V2

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net2.parameters(), lr=initial_lr, momentum=0.9)

for epoch in range(20):  # 데이터셋을 수차례 반복합니다.
    if epoch == 0:
        lr = initial_lr
    elif epoch % 2 == 0 and epoch != 0:
        lr *= 0.94
        optimizer = optim.SGD(net2.parameters(), lr=lr, momentum=0.9)
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device2), labels.to(device2)

        optimizer.zero_grad()

        outputs = net2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        """
        running_loss += loss.item()
        
        show_period = 250
        if i % show_period == show_period-1:    # print every "show_period" mini-batches
            print('[%d, %5d] loss: %.7f' %
                  (epoch + 1, i + 1, running_loss / show_period))
            running_loss = 0.0
        """
        
    #validation part
    correct = 0
    total = 0
    for i, data in enumerate(valid_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device2), labels.to(device2)
        outputs = net2(inputs)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('[%d epoch] Accuracy of the network on the validation images: %d %%' % 
          (epoch, 100 * correct / total)
         )

print('Finished Training')

[0 epoch] Accuracy of the network on the validation images: 74 %
[1 epoch] Accuracy of the network on the validation images: 77 %
[2 epoch] Accuracy of the network on the validation images: 79 %
[3 epoch] Accuracy of the network on the validation images: 80 %
[4 epoch] Accuracy of the network on the validation images: 81 %
[5 epoch] Accuracy of the network on the validation images: 81 %
[6 epoch] Accuracy of the network on the validation images: 83 %
[7 epoch] Accuracy of the network on the validation images: 81 %
[8 epoch] Accuracy of the network on the validation images: 82 %
[9 epoch] Accuracy of the network on the validation images: 83 %
[10 epoch] Accuracy of the network on the validation images: 84 %
[11 epoch] Accuracy of the network on the validation images: 85 %
[12 epoch] Accuracy of the network on the validation images: 83 %
[13 epoch] Accuracy of the network on the validation images: 84 %
[14 epoch] Accuracy of the network on the validation images: 85 %
[15 epoch] Accuracy 

In [26]:
dataiter = iter(test_loader)
images, labels = next(dataiter)
images, labels = images.to(device2), labels.to(device2)

outputs = net2(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(len(images))))

Predicted:    cat plane plane   cat   car plane plane  deer  bird  deer plane  deer   cat  deer   car  deer plane   cat  bird  deer  deer   cat   cat  bird   car  bird  deer  bird plane  bird   cat   cat


In [28]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device2), labels.to(device2)
        outputs = net2(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 86 %


In [30]:
class_correct = list(0. for i in range(5)) #changed from 10 to 2 classes
class_total = list(0. for i in range(5)) #changed from 10 to 2 classes
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device2), labels.to(device2)
        outputs = net2(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
                
        for i in range(labels.shape[0]):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(5): #changed from 10 to 2 classes
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 94 %
Accuracy of   car : 89 %
Accuracy of  bird : 75 %
Accuracy of   cat : 84 %
Accuracy of  deer : 89 %


In [31]:
import torch
import time
from sklearn.metrics import precision_score, recall_score, f1_score
correct = 0
total = 0
all_labels = []
all_predictions = []

# To measure inference time
start_time = time.time()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device2), labels.to(device2)
        
        # Forward pass to get outputs
        outputs = net2(images)
        _, predicted = torch.max(outputs.data, 1)

        # Collect all labels and predictions for precision, recall, F1 score calculation
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

        # Calculating correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate inference time
inference_time = time.time() - start_time

# Convert lists to numpy arrays for metric calculation
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

# Calculate precision, recall, and F1 score
precision = precision_score(all_labels, all_predictions, average='macro')
recall = recall_score(all_labels, all_predictions, average='macro')
f1 = f1_score(all_labels, all_predictions, average='macro')

print('Accuracy of the network on the test images: {:.2f} %'.format(100 * correct / total))
print('Precision: {:.4f}'.format(precision))
print('Recall: {:.4f}'.format(recall))
print('F1 Score: {:.4f}'.format(f1))
print('Inference Time: {:.2f} seconds'.format(inference_time))

Accuracy of the network on the test images: 86.70 %
Precision: 0.8717
Recall: 0.8670
F1 Score: 0.8668
Inference Time: 25.80 seconds
